## Lab #6 Assignment
## QMSS S5019 - Data Analysis with Python
## Lovina Putri
## CUID: lap2236

***(1) Set-up packages and code***

In [194]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf

***(2) Import data***

In [195]:
url = 'https://www.qogdata.pol.gu.se/data/qog_bas_cs_jan25.xlsx'
df = pd.read_excel(url)

***(3) Data check***

In [196]:
df.head()

,cname_qog,cname,ccode_qog,ccodecow,ccodealp,ccode,ajr_settmort,atop_ally,atop_number,bci_bci,...,wvs_imprel,wvs_pmi12,wvs_psarmy,wvs_psdem,wvs_psexp,wvs_pssl,wvs_relacc,wvs_satfin,wvs_subh,wvs_trust
0,Afghanistan,Afghanistan,4,700.0,AFG,4.0,4.540098,1.0,1.0,70.507206,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Albania,Albania,8,339.0,ALB,8.0,NaN,1.0,8.0,56.955081,...,2.869328,NaN,1.596485,3.849031,3.475513,1.744196,NaN,NaN,3.488758,0.027857
2,Algeria,Algeria,12,615.0,DZA,12.0,4.359270,1.0,9.0,49.363506,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Andorra,Andorra,20,232.0,AND,20.0,NaN,1.0,2.0,14.475297,...,2.034930,2.710394,1.336049,3.681363,2.635721,1.830491,1.751004,6.561316,4.089641,0.255744
4,Angola,Angola,24,540.0,AGO,24.0,5.634789,1.0,8.0,62.067964,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


***(4) Questions***

**1 -- Run a multiple linear probability model (have at least 2 Xs in the model). Tell me how you think your independent variables will affect your dependent variable. Interpret your results. Were your expectations correct? Why or why not?**  

Global Competitiveness Index is (wef_gci) the dependent variable. The data range from 1 to 100 with 1 is low competitiveness and 100 is high competitiveness. This has been recoded to binary so low competitiveness (1-60) is 0 and high competitiveness (60-100) is 1. I choose 60 as the cut-off since the lowest vaue of GCI is around 35.

I believe that a better economic competitiveness should be associated with a higher innovation capability (wef_ci), higher HDI (undp_hdi), higher property rights (wef_pr), higher government's anti-corruption policy (bti_acp), and higher economic freedom (fi_index).

In [197]:
# Normalize HDI, FI, and PR to 0-100
df["undp_hdi_norm"] = df["undp_hdi"] * 100
df["wef_pr_norm"] = df["wef_pr"] * 100/7
df["fi_index_norm"] = df["fi_index"] * 100/10
df["bti_acp_norm"] = df["bti_acp"] * 100/10

sub = df.dropna(subset = ['wef_gci', 'wef_ci', 'undp_hdi_norm', 'wef_pr_norm', 'bti_acp_norm', 'fi_index_norm'])

In [198]:
conditions = [(sub['wef_gci'] < 60) , (sub['wef_gci'] > 60)]
choices = [0, 1]
sub['hi_competitive'] = np.select(conditions, choices, default=np.nan)

/tmp/ipython-input-198-4052616556.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub['hi_competitive'] = np.select(conditions, choices, default=np.nan)


In [199]:
pd.crosstab(index=sub["hi_competitive"], columns="count")

col_0,count
hi_competitive,
0.0,63
1.0,46


In [200]:
lm1 = smf.ols(formula = 'hi_competitive ~ wef_ci + undp_hdi_norm + wef_pr_norm + bti_acp_norm + fi_index_norm', data = sub).fit()
print (lm1.summary())

                            OLS Regression Results                            
Dep. Variable:         hi_competitive   R-squared:                       0.621
Model:                            OLS   Adj. R-squared:                  0.603
Method:                 Least Squares   F-statistic:                     33.80
Date:                Tue, 01 Jul 2025   Prob (F-statistic):           2.74e-20
Time:                        16:36:58   Log-Likelihood:                -24.846
No. Observations:                 109   AIC:                             61.69
Df Residuals:                     103   BIC:                             77.84
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -1.7730      0.241     -7.359

1. Innovation capability (wef_cu) is positively associated with economic competitiveness, and the association is statistically significant.

2. Human Development Index (undp_hdi) is positively associated with economic competitiveness and is statistically significant.

3. Property rights (wef_pr) is positively associated with a economic competitiveness, and the association is not statistically significant.

4. Anti-corruption policy (bti_acp) is positively associated with economic competitiveness and not statistically significant.

5. Economic freedom index (fi_index) is positively associated with a economic competitiveness and has a statistically significant association.


As expected, with better economic competitiveness, innovation capability, HDI, economic freedom index, anti-corruption policy, and property rights are higher. But the property rights and anti-corruption policy do not appear to indicate the performance of economic competitiveness.

**2 -- Run a multiple (binary) logistic model. (It can be the same as the above LPM or a new model.) If it is a new model, tell me how you think your independent variables will affect your dependent variable. Interpret your results in the logit scale. Were your expectations correct? Why or why not?**

In [201]:
logit1 = sm.formula.logit(formula = 'hi_competitive ~ wef_ci + undp_hdi_norm + wef_pr_norm + bti_acp_norm + fi_index_norm', data = sub).fit()
print (logit1.summary())

Optimization terminated successfully.
         Current function value: 0.107619
         Iterations 12
                           Logit Regression Results                           
Dep. Variable:         hi_competitive   No. Observations:                  109
Model:                          Logit   Df Residuals:                      103
Method:                           MLE   Df Model:                            5
Date:                Tue, 01 Jul 2025   Pseudo R-squ.:                  0.8420
Time:                        16:36:58   Log-Likelihood:                -11.730
converged:                       True   LL-Null:                       -74.222
Covariance Type:            nonrobust   LLR p-value:                 2.759e-25
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept       -78.0718     29.576     -2.640      0.008    -136.040     -20.104
wef_ci            0

Logit model shows the same results in terms of the direction of the coefficients. However, the significance level is different -- in this model, all variables are statistically significant in 5% significance level.

1. Innovation capability (wef_cu) is positively associated with economic competitiveness, and the association is statistically significant.

2. Human Development Index (undp_hdi) has the largest coefficient, and positively associated with economic competitiveness and is statistically significant.

3. Property rights (wef_pr) is positively associated with a economic competitiveness, and the association is statistically significant (in previous model, this variable is not statistically significant at 5% level).

4. Anti-corruption policy (bti_acp) is positively associated with economic competitiveness and still not statistically significant at 5% level.

5. Economic freedom index (fi_index) is positively associated with a economic competitiveness and has a statistically significant association.

**3 -- Get odds ratios from your logit model in Question 2 and interpret some of them.**

In [202]:
np.exp(logit1.params)

,0
Intercept,1.241222e-34
wef_ci,1.840688e+00
undp_hdi_norm,1.288083e+00
wef_pr_norm,1.246728e+00
bti_acp_norm,1.082137e+00
fi_index_norm,1.361457e+00


* For each one-unit increase in innovation capacity, the odds of the a country considered to have a high competitiveness increase by a factor of 84.1%.

* For each one-unit increase in HDI, the odds of the a country considered to have a high competitiveness increase by a factor of 28.8%.

* For each one-unit increase in the property rights, the odds of the economy being considered good rise by a factor of 24.7%.

* For each one-unit increase in anti-corruption policy, the odds of the a country considered to have a high competitiveness increase by a factor of 8.2%.

* For each one-unit increase in economic openness, the odds of the a country considered to have a high competitiveness increase by a factor of 36.1%.

**4 -- Get predicted probabilities from your logit model in Question 2 for some constellations of X values and interpret the results.**

In [203]:
def logit2prob (logit):
    odds = np.exp(logit)
    prob = odds / (1 + odds)
    return(prob);

intercept = logit1.params.Intercept
b_wef_ci = logit1.params.wef_ci
b_undp_hdi_norm = logit1.params.undp_hdi_norm
b_wef_pr_norm = logit1.params.wef_pr_norm
b_bti_acp_norm = logit1.params.bti_acp_norm
b_fi_index_norm = logit1.params.fi_index_norm

wef_ci = 50, undp_hdi_norm = 60, wef_pr_norm = 60, bti_acp_norm = 10, fi_index_norm = 60

53.9% chance of economy being highly competitive

In [204]:
logits_exh = intercept + (b_wef_ci * 50) + (b_undp_hdi_norm * 60) + (b_wef_pr_norm * 60) + (b_bti_acp_norm * 10) + (b_fi_index_norm * 60)
logit2prob(logits_exh)

np.float64(0.5395558784200933)

wef_ci = 60, undp_hdi_norm = 50, wef_pr_norm = 60, bti_acp_norm = 50, fi_index_norm = 50

97.8% chance of economy being highly competitive

In [205]:
logits_exh = intercept + (b_wef_ci * 60) + (b_undp_hdi_norm * 50) + (b_wef_pr_norm * 60) + (b_bti_acp_norm * 50) + (b_fi_index_norm * 50)
logit2prob(logits_exh)

np.float64(0.9781259486855765)

Probability of a country becoming highly competitive with average values on the X

16.1% chance of economy being highly competitive

In [206]:
logits_exh = intercept + (sub.wef_ci.mean() * b_wef_ci) + (sub.undp_hdi_norm.mean() * b_undp_hdi_norm) + (sub.wef_pr_norm.mean() * b_wef_pr_norm) + (sub.bti_acp_norm.mean() * b_bti_acp_norm) + (sub.fi_index_norm.mean() * b_fi_index_norm)
logit2prob(logits_exh)

np.float64(0.16085926286412083)